# Tool Call Guardrails

Tool call guardrails allow you to inspect and validate the **arguments** of tool calls that agents suggest before they are executed. This provides a safety mechanism to detect potentially harmful, illegal, or policy-violating tool arguments.

**Key Features:**
- Inspects tool name and arguments extracted from agent messages
- Uses LLM-based detection for context-aware validation
- Can be used as input or output guardrails
- Automatically handles cases where no tool calls are present

Please find documentation about guardrails [here](https://docs.ag2.ai/latest/docs/user-guide/advanced-concepts/orchestration/group-chat/guardrails).

````{=mdx}
:::info Requirements
Install `ag2`:
```bash
pip install ag2[openai]
```

For more information, please refer to the [installation guide](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/installing-ag2).
:::
````

## Set up LLM Configuration

Configure your LLM for both the agents and the guardrail checks.

In [ ]:
import os

from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent
from autogen.agentchat.group.guardrails import ToolCallLLMGuardrail
from autogen.agentchat.group.targets.transition_target import StayTarget
from autogen.llm_config import LLMConfig

# Configure LLM for agents and guardrails
llm_config = LLMConfig(
    model="gpt-5-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_type="openai",
)

## Example 1: Output Guardrail - Checking Tool Calls Before Execution

In this example, we register a `ToolCallLLMGuardrail` as an **output guardrail** on an assistant. When the assistant suggests tool calls, the guardrail inspects the tool arguments before they are executed.

The guardrail will:
- Extract tool call information (name and arguments) from the assistant's reply
- Send this information to an LLM with your condition
- Return `activated=True` if the condition is met (e.g., harmful content detected)
- Return `activated=False` if no tool calls are present or the condition is not met

In [ ]:
# Create a tool call guardrail
tool_call_guardrail = ToolCallLLMGuardrail(
    name="tool_safety",
    condition="arguments contain harmful or illegal content.",
    target=StayTarget(),
    llm_config=llm_config,
    activation_message="Tool call blocked: Potentially harmful arguments detected.",
)

# Create an assistant with the guardrail
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="You are a helpful assistant. Use tools when needed.",
)

# Register the guardrail as an output guardrail
assistant.register_output_guardrail(tool_call_guardrail)

print(f"Guardrail condition: {tool_call_guardrail.condition}")

### Understanding the Guardrail Condition

Notice that `ToolCallLLMGuardrail` automatically prepends "Here are arguments to a Tool call function." to your condition. This helps the LLM understand the context of what it's checking.

## Example 2: Using Tool Call Guardrail in GroupChat

Let's create a complete example with a GroupChat where an assistant can suggest tool calls, and the guardrail checks them.

In [ ]:
# Create a guardrail for checking tool arguments
joke_guardrail = ToolCallLLMGuardrail(
    name="no_jokes",
    condition="arguments contain jokes or humor.",
    target=StayTarget(),
    llm_config=llm_config,
    activation_message="⚠️ Joke detected in tool arguments - request blocked.",
)

def joke_tool(joke: str) -> str:
    """
        This tool is used to detect PII in the code.
    """
    return joke

# Create agents
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    functions=[joke_tool]
)
assistant.register_input_guardrail(joke_guardrail)

# Register function for execution so GroupChat can find an agent to execute it
assistant.register_for_execution()(joke_tool)

user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": False},
    human_input_mode="NEVER",
)

# Create GroupChat
groupchat = GroupChat(
    agents=[assistant, user_proxy],
    messages=[],
    max_round=3,
)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Start a conversation
user_proxy.run(
    manager,
    message="write a joke, use joke tool to write a joke",
).process()

## Example 3: Direct Guardrail Check

You can also call the guardrail's `check()` method directly with a context that contains tool calls. This is useful for testing or programmatic validation.

In [ ]:
from typing import Any

# Create a guardrail
guardrail = ToolCallLLMGuardrail(
    name="safety_check",
    condition="arguments contain dangerous or illegal content.",
    target=StayTarget(),
    llm_config=llm_config,
)

# Simulate a context with tool calls
context_with_tool_calls: list[dict[str, Any]] = [
    {"role": "user", "content": "Run this code for me."},
    {
        "role": "assistant",
        "content": None,
        "tool_calls": [
            {
                "id": "call_1",
                "type": "function",
                "function": {"name": "run_code", "arguments": '{"code": "1 + 1"}'},
            }
        ],
    },
]

# Check the tool calls
result = guardrail.check(context=context_with_tool_calls)

print(f"Guardrail activated: {result.activated}")
print(f"Justification: {result.justification}")
print(f"Guardrail name: {result.guardrail.name}")

## Example 4: Input Guardrail

You can also use `ToolCallLLMGuardrail` as an **input guardrail** to check tool calls before they reach an agent. This is useful when you want to validate tool calls from one agent before another agent processes them.

In [ ]:
# Create an input guardrail
input_guardrail = ToolCallLLMGuardrail(
    name="input_safety",
    condition="arguments contain commands that could delete files or modify system settings.",
    target=StayTarget(),
    llm_config=llm_config,
    activation_message="🚫 Dangerous tool call detected - blocked.",
)

# Register as input guardrail on an executor agent
executor = AssistantAgent(
    name="executor",
    llm_config=llm_config,
)
executor.register_input_guardrail(input_guardrail)

print("Input guardrail registered on executor agent")

## How It Works

1. **Extraction**: When `check()` is called, `ToolCallLLMGuardrail` extracts tool call information from the context:
   - If context is a list of messages, it checks the last message for `tool_calls`
   - If context is a dict, it checks for a `tool_calls` key
   - If no tool calls are found, it returns `activated=False` immediately

2. **Parsing**: Tool calls are parsed into a string format: `name='tool_name' arguments='{...}'`

3. **LLM Check**: The parsed tool call data is sent to the LLM with your condition (prefixed with "Here are arguments to a Tool call function.")

4. **Result**: The LLM returns whether the condition is met, and a `GuardrailResult` is returned with `activated`, `justification`, and the `guardrail` reference.

## Best Practices

- Use **output guardrails** when you want to check tool calls before they are executed
- Use **input guardrails** when you want to validate tool calls before another agent processes them
- Choose appropriate **transition targets** (e.g., `StayTarget`, `AgentTarget`) based on your workflow
- Provide clear **activation messages** to help users understand why a guardrail was triggered
- Test your guardrails with various tool call scenarios to ensure they work as expected